### Используем НОРМАЛЬНЫЙ апи клиент, который по хорошему надо бы интегрировать в дансвер

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
NOTION_API_KEY = os.getenv("NOTION_API_KEY")
NOTION_ROOT_ID = os.getenv("NOTION_ROOT_ID")
NOTION_DATABASE_ID = os.getenv("NOTION_DATABASE_ID")

In [ ]:
from notion_client import Client, AsyncClient

# Initialize a client with the integration token
notion = Client(auth=NOTION_API_KEY)


In [ ]:
page_id = NOTION_ROOT_ID

# Retrieve the page
page = notion.pages.retrieve(page_id=page_id)

# Print page properties
print(page["properties"])


In [4]:

pages = notion.databases.query(database_id=NOTION_DATABASE_ID) # корретно влзвращает и страницы и теги, но нужно не проморгать пагинацию (has_more и next_cursor)

In [5]:
page = pages['results'][1]

In [6]:
tag_groups = {}
for page in pages['results']:
    for property in page['properties']:
        if page['properties'][property]['type'] in ["select", "multi_select"]:
            if not property in tag_groups:
                tag_groups[property] = set()
            info = page['properties'][property]
            if info['type'] == 'select':
                tag_groups[property].add(info['select']['name'])
            else:
                tag_groups[property].update([x['name'] for x in info['multi_select']])



In [7]:
remove_list_for_tags = [
    "115",
    "18-25",
    "AMELI",
    "API",
    "APL",
    "APS",
    "CADA",
    "CAF",
    "CIR",
    "CPAM",
    "CROUS",
    "CSS",
    "CVEC",
    "CritAir",
    "EHPAD",
    "FSL",
    "HUDA",
    "MDPH",
    "OFII",
    "OFPRA",
    "OMC",
    "PACS",
    "PASS",
    "PUMa",
    "RSA",
    "SPADA",
    "VAE",
    "VAPP",
    "VES",
    "ZAPI",
    "asso",
]

In [8]:
tag_groups['Tags'] = tag_groups['Tags'] - set(remove_list_for_tags)

In [ ]:
tag_groups

In [10]:
# not transform to this format {"dtags" : [{"tag_key": "Tags", "all_possible_tag_values" : ["равенство","женералист"]}]}
tag_groups_formatted = [{"tag_key": key, "all_possible_tag_values": list(value)} for key, value in tag_groups.items()]

In [11]:
# dump to single string json compact as possible
import json
tag_groups_formatted_json = json.dumps({"dtags":tag_groups_formatted}, separators=(',', ':'), ensure_ascii=False)
with open('tag_groups.json', 'w') as f:
    f.write(tag_groups_formatted_json)

In [ ]:
page['properties']['Select']['select']['name']

In [ ]:
page["url"]

In [ ]:
page["properties"]["URL"]["url"]

In [15]:
id_to_tags = {}
for page in pages['results']:
    ms_dicts = page['properties']['Tags']['multi_select']
    tag_names = [tag["name"] for tag in ms_dicts]
    id_to_tags[page['id']] = tag_names

In [ ]:
pages

In [ ]:
id_to_tags["f2b1db66-7b1d-4e1d-8c4c-3c8ceb481511"]

In [ ]:
notion.pages.retrieve(page_id=database_id)

In [ ]:
has_more : bool = pages["has_more"]
cursor : str = pages["next_cursor"]

In [ ]:
pages['next_cursor']

In [ ]:
database_id = NOTION_DATABASE_ID
notion.databases.retrieve(database_id=database_id)

In [ ]:
def retrieve_blocks(page_id):
    all_blocks = []
    start_cursor = None
    while True:
        response = notion.blocks.children.list(block_id=page_id, start_cursor=start_cursor)
        blocks = response.get("results", [])
        all_blocks.extend(blocks)
        if not response.get("has_more", False):
            break
        start_cursor = response.get("next_cursor", None)
    return all_blocks

root_page_id = "a283ff3a46804b39b50e29c566369c2f"

# Retrieve blocks from the root page
blocks = retrieve_blocks(root_page_id)
visited_ids = set()

def retrieve_all_blocks(blocks, level=0):
    all_blocks = []
    for block in blocks:
        if block["id"] not in visited_ids:
            #print("  " * level + block["type"], block.get("id", ""))
            all_blocks.append(block)
            visited_ids.add(block["id"])
            if block.get("has_children", False):
                child_blocks = retrieve_blocks(block["id"])
                new_blocks = retrieve_all_blocks(child_blocks, level + 1)
                all_blocks.extend(new_blocks) if new_blocks else None
    return all_blocks

# Start retrieving all nested blocks
all_blocks = retrieve_all_blocks(blocks)


In [ ]:
for i, block in enumerate(all_blocks):
    if "https://www.defenseurdesdroits.fr/carte-des-delegues" in str(block):
        print(block)
        print(i)


In [ ]:
print(all_blocks[131])

### Используем то что уже зашито в дасвер, чтобы ничего не сломать

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
#NOTION_API_KEY = os.getenv("NOTION_API_KEY")
NOTION_API_KEY = os.getenv("NOT_NEW_KEY")
NOTION_ROOT_ID = os.getenv("NOTION_ROOT_ID")
NOTION_DATABASE_ID = os.getenv("NOTION_DATABASE_ID")

In [2]:
from onyx.connectors.notion.connector import NotionConnector

In [ ]:
#connector = NotionConnector(root_page_id="f2c93c208bd345038ef7e99f9db7b23e") # just one page here
connector = NotionConnector(root_page_id=NOTION_DATABASE_ID) # database with pages and tags
connector.load_credentials({"notion_integration_token": NOTION_API_KEY})
connector.is_db

In [4]:
page_properties_example = {
    "Tags": {
        "id": "BJSo",
        "type": "multi_select",
        "multi_select": [
            {
                "id": "3d4c675d-91b5-4fcd-877c-f7f8ac453733",
                "name": "машина",
                "color": "yellow",
            },
            {
                "id": "a606a9b2-2a8b-49e7-8713-409df9b3482e",
                "name": "покупка",
                "color": "default",
            },
            {
                "id": "e19d2072-e62a-4cb4-9545-f1e288202efb",
                "name": "immatriculation",
                "color": "orange",
            },
            {
                "id": "d342c1e4-3dc2-4977-81ee-46629dc2f6b2",
                "name": "CVEC",
                "color": "blue",
            },
        ],
    },
    "Last edited time 1": {
        "id": "PRCm",
        "type": "last_edited_time",
        "last_edited_time": "2024-11-21T09:05:00.000Z",
    },
    "Verification": {
        "id": "TEMG",
        "type": "verification",
        "verification": {"state": "unverified", "verified_by": None, "date": None},
    },
    "Select": {
        "id": "%5EClT",
        "type": "select",
        "select": {
            "id": "75a4353a-5741-4800-8a27-d62e67dd167b",
            "name": "Если вы во Франции",
            "color": "pink",
        },
    },
    "Last edited time": {
        "id": "gW%3D%7D",
        "type": "last_edited_time",
        "last_edited_time": "2024-11-21T09:05:00.000Z",
    },
    "URL": {"id": "h%5D%3D%3E", "type": "url", "url": None},
    "Multiselect": {
        "id": "lSKn",
        "type": "multi_select",
        "multi_select": [
            {
                "id": "d2f4753d-3123-467e-8f0a-42fa78e949c6",
                "name": "Машина",
                "color": "orange",
            }
        ],
    },
    "Page": {
        "id": "title",
        "type": "title",
        "title": [
            {
                "type": "mention",
                "mention": {
                    "type": "page",
                    "page": {"id": "53b2b625-c860-4e55-a285-e42d11a7adea"},
                },
                "annotations": {
                    "bold": False,
                    "italic": False,
                    "strikethrough": False,
                    "underline": False,
                    "code": False,
                    "color": "default",
                },
                "plain_text": "Untitled",
                "href": "https://www.notion.so/53b2b625c8604e55a285e42d11a7adea",
            },
            {
                "type": "text",
                "text": {"content": " ", "link": None},
                "annotations": {
                    "bold": False,
                    "italic": False,
                    "strikethrough": False,
                    "underline": False,
                    "code": False,
                    "color": "default",
                },
                "plain_text": " ",
                "href": None,
            },
        ],
    },
    "Owner": {
        "id": "notion%3A%2F%2Fwiki%2Fowner_property",
        "type": "people",
        "people": [
            {
                "object": "user",
                "id": "f91b5190-55f4-4f8b-8539-db48cdabb114",
                "name": "Exil Solidaire",
                "avatar_url": "https://s3-us-west-2.amazonaws.com/public.notion-static.com/e2e4a91a-7c8a-41be-a520-2bf991757245/favicon-32x32.png",
                "type": "person",
                "person": {"email": "exil-solidaire@proton.me"},
            }
        ],
    },
}

In [ ]:
class DummyPage:
    def __init__(self, properties = None):
        self.properties = properties
    def _extract_source_id_from_href_page(page):
        try:
            title_objects = page.properties.get('Page', {}).get('title', [])
            for obj in title_objects:
                if obj.get('type') == 'mention' and 'mention' in obj:
                    return obj['mention'].get('page', {}).get('id')
            return None
        except (AttributeError, KeyError):
            return None
dummy_page = DummyPage(page_properties_example)
print(dummy_page._extract_source_id_from_href_page())

In [ ]:
connector._fetch_page("1423888dae8d80e5acbcda8d5412a73a")

In [ ]:
out = [_ for _ in connector.load_from_state()]

In [ ]:
for doc_batch in out:
    for doc in doc_batch:
        print(doc)

In [ ]:
pages = [connector._fetch_page("a283ff3a46804b39b50e29c566369c2f")]

In [ ]:
out

In [ ]:
parsed_pages = []
for parsed_page in connector._read_pages(pages):
    parsed_pages.append(parsed_page)

In [ ]:
for section in parsed_pages[0]:
    print(section)